# 2025 DATA·AI 분석 경진대회 - 논문·데이터 추천 에이전트 평가

이 노트북은 LLM 기반 연구 데이터/논문 추천 시스템의 성능을 평가합니다.

**실행 환경**
- GPU: NVIDIA RTX 3080 이상
- CUDA: 11.8+
- Python: 3.10+

## 1. 환경 설정 및 라이브러리 임포트

In [1]:
import sys
import os
import json
import logging
from datetime import datetime
from pathlib import Path
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

# 프로젝트 루트 경로
project_root = '/home/infidea/backup-data/paper-reco-agent'
sys.path.insert(0, project_root)

print(f"프로젝트 루트: {project_root}")

# 로깅 설정
os.makedirs(os.path.join(project_root, 'logs'), exist_ok=True)

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout),
        logging.FileHandler(os.path.join(project_root, 'logs/evaluation.log'))
    ],
    force=True
)

print("✅ 로깅 설정 완료")

# 평가 모듈 임포트
from src.evaluation.metrics import (
    evaluate_recommendations,
    batch_evaluate,
    format_evaluation_report
)
from src.config.settings import settings

print("✅ 평가 모듈 임포트 완료")

프로젝트 루트: /home/infidea/backup-data/paper-reco-agent
✅ 로깅 설정 완료
✅ 평가 모듈 임포트 완료


/home/infidea/backup-data/miniconda3/envs/paper-agent/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. 테스트 데이터셋 로드

In [2]:
# 테스트 데이터셋 로드
testset_path = os.path.join(project_root, 'data', 'test', 'testset_aug.json')

with open(testset_path, 'r', encoding='utf-8') as f:
    testset = json.load(f)

print(f"✅ 테스트 데이터셋 로드 완료: {testset_path}")
print(f"📊 테스트 카테고리 수: {len(testset.keys())}")

# 모든 테스트 케이스 추출
all_test_cases = []
for category, cases in testset.items():
    for case in cases:
        case['category'] = category
        all_test_cases.append(case)

print(f"📝 총 테스트 케이스: {len(all_test_cases)}개")
print(f"\n첫 번째 테스트 케이스 예시:")
print(json.dumps(all_test_cases[0], indent=2, ensure_ascii=False)[:500] + "...")

✅ 테스트 데이터셋 로드 완료: /home/infidea/backup-data/paper-reco-agent/data/test/testset_aug.json
📊 테스트 카테고리 수: 2
📝 총 테스트 케이스: 13개

첫 번째 테스트 케이스 예시:
{
  "input_dataset": {
    "svc_id": "c1f03ab868f32371ec97a650bcdf39ac",
    "title": "한반도 기상 관측 데이터 세트",
    "description": "한반도 종관 기상 관측 세스템으로부터의 관측 데이터 세트",
    "keywords": [
      "한반도",
      "기상",
      "센서데이터"
    ]
  },
  "candidate_pool": {
    "papers": [
      {
        "id": "JAKO202029757857862",
        "title": "시군구별 대기오염 및 기상 데이터",
        "abstract": "대기오염의 사회경제적 효과에 대한 연구에는 측정된 대기오염 물질, 기상 자료, 그리고 사회경제적 데이터의 병합이 필요하다. 이들 자료들의 시간적&#x00B7;공간적 범위와 단위가 상이하기 때문에 분석에 필요한 데이터 가공에 많은 시...


### 2.1 테스트셋 데이터 분포 분석

In [3]:
# svd_id(=svc_id/id)별 candidate_pool 분석 (type 컬럼 기반 2행 구조)
import pandas as pd
from pathlib import Path
from IPython.display import display

# ---- Helper ----
LEVEL_TO_REL = {"강추": 2, "추천": 1, "참고": 0}

def get_svd_id(d):
    return d.get("svd_id") or d.get("svc_id") or d.get("id") or d.get("dataset_id")

def ensure_relevance_score(items):
    items = items or []
    out = []
    for it in items:
        it = dict(it)
        if "relevance_score" not in it:
            it["relevance_score"] = LEVEL_TO_REL.get(it.get("level"), 0)
        out.append(it)
    return out

def count_scores(items):
    total = len(items)
    s2 = sum(1 for x in items if x.get("relevance_score", 0) == 2)
    s1 = sum(1 for x in items if x.get("relevance_score", 0) == 1)
    s0 = sum(1 for x in items if x.get("relevance_score", 0) == 0)
    def ratio(x): 
        return (x / total * 100.0) if total else 0.0
    return {
        "total": total,
        "score2_cnt": s2, "score1_cnt": s1, "score0_cnt": s0,
        "score2_ratio(%)": round(ratio(s2), 1),
        "score1_ratio(%)": round(ratio(s1), 1),
        "score0_ratio(%)": round(ratio(s0), 1),
    }

# ---- Build rows (paper / dataset 각각 한 행씩) ----
rows = []
for case in all_test_cases:
    category = case.get("category")
    ds = case.get("input_dataset", {})
    svd_id = get_svd_id(ds)
    title = ds.get("title", "")
    pool = case.get("candidate_pool", {}) or {}

    for tname, items in {"paper": pool.get("papers", []), "dataset": pool.get("datasets", [])}.items():
        items = ensure_relevance_score(items or [])
        stats = count_scores(items)
        rows.append({
            "category": category,
            "svd_id": svd_id,
            "title": title,
            "type": tname,
            **stats
        })

# ---- DataFrame ----
df_svd_type = pd.DataFrame(rows).sort_values(["category", "svd_id", "type"]).reset_index(drop=True)

print("✅ svd_id 단위 분석 테이블 (type 컬럼 기반):")
display(df_svd_type)

✅ svd_id 단위 분석 테이블 (type 컬럼 기반):


,category,svd_id,title,type,total,score2_cnt,score1_cnt,score0_cnt,score2_ratio(%),score1_ratio(%),score0_ratio(%)
0,국내,000e4340354d8219bdb3af6a7ad4800d,이동형 도시환경 센서 및 도로영상 데이터 셋,dataset,5,2,3,0,40.0,60.0,0.0
1,국내,000e4340354d8219bdb3af6a7ad4800d,이동형 도시환경 센서 및 도로영상 데이터 셋,paper,5,2,2,1,40.0,40.0,20.0
2,국내,33da0205f56477ba41dec7d7db3258a6,Dataset for Intervention strategies against CO...,dataset,5,1,2,2,20.0,40.0,40.0
3,국내,33da0205f56477ba41dec7d7db3258a6,Dataset for Intervention strategies against CO...,paper,5,1,2,2,20.0,40.0,40.0
4,국내,4fc87916027f5568850d33ecd2f7680f,한국인 정상인 심전도 세부파형,dataset,5,4,1,0,80.0,20.0,0.0
5,국내,4fc87916027f5568850d33ecd2f7680f,한국인 정상인 심전도 세부파형,paper,5,2,2,1,40.0,40.0,20.0
6,국내,8d498db96633f07fe1d82ebe43daaf45,Landsat-5/-7/-8 및 Sentinel-2 광학 위성 영상을 활용한 천지호...,dataset,5,1,2,2,20.0,40.0,40.0
7,국내,8d498db96633f07fe1d82ebe43daaf45,Landsat-5/-7/-8 및 Sentinel-2 광학 위성 영상을 활용한 천지호...,paper,5,2,2,1,40.0,40.0,20.0
8,국내,a59a0a7d0c894a81a2936833b0790e0a,의료영상데이터,dataset,5,3,2,0,60.0,40.0,0.0
9,국내,a59a0a7d0c894a81a2936833b0790e0a,의료영상데이터,paper,5,2,2,1,40.0,40.0,20.0


## 3. 추천 에이전트 초기화

In [4]:
# 추천 에이전트 임포트 및 초기화
from src.agents.recommendation_agent import KoreanResearchRecommendationAgent
from src.config.settings import settings

print("모델 설정:")
print(f"  - 모델명: {settings.MODEL_NAME}")
print(f"  - 임베딩 모델: {settings.EMBEDDING_MODEL}")
print(f"  - 개발 모드: {settings.DEV_MODE}")
print("\n🚀 에이전트 초기화 중... (수 분 소요될 수 있습니다)")

agent = KoreanResearchRecommendationAgent()

print("\n✅ 에이전트 초기화 완료")
print(f"모델 정보: {json.dumps(agent.llm_model.get_model_info(), indent=2, ensure_ascii=False)}")

2025-10-16 07:22:19,543 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device_name: cuda:0
2025-10-16 07:22:19,545 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: intfloat/multilingual-e5-large
모델 설정:
  - 모델명: google/gemma-2-9b-it
  - 임베딩 모델: intfloat/multilingual-e5-large
  - 개발 모드: False

🚀 에이전트 초기화 중... (수 분 소요될 수 있습니다)
2025-10-16 07:22:26,622 - src.agents.recommendation_agent - INFO - 🚀 프로덕션 모드로 실행: 실제 LLM 모델 사용
2025-10-16 07:22:26,624 - src.models.llm_model - INFO - 🚀 Gemma 모델 로딩 시작: google/gemma-2-9b-it
2025-10-16 07:22:26,625 - src.models.llm_model - INFO -    - 디바이스: cuda
2025-10-16 07:22:27,868 - src.models.llm_model - INFO -    - FP16 모드


`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 4/4 [00:18<00:00,  4.50s/it]


2025-10-16 07:23:00,260 - src.models.llm_model - INFO - ✅ Gemma 모델 로딩 완료

✅ 에이전트 초기화 완료
모델 정보: {
  "model_name": "google/gemma-2-9b-it",
  "model_type": "Gemma",
  "device": "cuda",
  "dtype": "float16",
  "max_tokens_reranking": 1024,
  "max_tokens_keyword": 300,
  "max_tokens": 1024,
  "temperature": 0.0,
  "parameters": "9B",
  "context_length": "8K"
}


## 4. 평가 실행

모든 테스트 케이스에 대해 추천을 생성하고 평가합니다.

In [5]:
# TODO: 추천 개수 설정 (선택사항)
# 최대값: agent의 max_paper_candidates, max_dataset_candidates 값
num_paper_recommendations = 5
num_dataset_recommendations = 5

# agent에서 최대 후보 개수 가져오기
MAX_PAPER_CANDIDATES = agent.max_paper_candidates
MAX_DATASET_CANDIDATES = agent.max_dataset_candidates

print(f"최대 추천 가능 개수: 논문 {MAX_PAPER_CANDIDATES}개, 데이터셋 {MAX_DATASET_CANDIDATES}개")

# 추천 개수 검증
if num_paper_recommendations > MAX_PAPER_CANDIDATES:
    print(f"⚠️  경고: 논문 추천 개수({num_paper_recommendations})가 최대값({MAX_PAPER_CANDIDATES})을 초과합니다.")
    print(f"   자동으로 {MAX_PAPER_CANDIDATES}개로 조정됩니다.")
    num_paper_recommendations = MAX_PAPER_CANDIDATES

if num_dataset_recommendations > MAX_DATASET_CANDIDATES:
    print(f"⚠️  경고: 데이터셋 추천 개수({num_dataset_recommendations})가 최대값({MAX_DATASET_CANDIDATES})을 초과합니다.")
    print(f"   자동으로 {MAX_DATASET_CANDIDATES}개로 조정됩니다.")
    num_dataset_recommendations = MAX_DATASET_CANDIDATES

if num_paper_recommendations < 1:
    print(f"⚠️  경고: 논문 추천 개수는 최소 1개 이상이어야 합니다.")
    num_paper_recommendations = 1

if num_dataset_recommendations < 1:
    print(f"⚠️  경고: 데이터셋 추천 개수는 최소 1개 이상이어야 합니다.")
    num_dataset_recommendations = 1

print(f"✅ 추천 설정: 논문 {num_paper_recommendations}개, 데이터셋 {num_dataset_recommendations}개")

최대 추천 가능 개수: 논문 10개, 데이터셋 10개
✅ 추천 설정: 논문 5개, 데이터셋 5개


In [6]:
# 평가 실행
all_eval_results = []
k_values = [3, 5]

# 추천 원본 결과(recommend_result.json 저장용)
all_recommendations = []

print(f"🔍 {len(all_test_cases)}개 테스트 케이스 평가 시작")
print(f"📏 평가 k 값: {k_values}")
print("=" * 80)

# 모델/임베딩 정보
try:
    _model_info = agent.llm_model.get_model_info()
except Exception:
    _model_info = {}
_model_info_safe = {
    "model_name": _model_info.get("model_name"),
    "model_type": _model_info.get("model_type"),
    "device": _model_info.get("device"),
    "dtype": _model_info.get("dtype"),
    "max_tokens": _model_info.get("max_tokens"),
    "temperature": _model_info.get("temperature"),
    "parameters": _model_info.get("parameters"),
    "context_length": _model_info.get("context_length"),
}

_embedding_info_safe = {
    "embedding_model": getattr(settings, "EMBEDDING_MODEL", None),
    "paper_hybrid_weights": {
        "alpha": getattr(settings, "PAPER_HYBRID_ALPHA", None),
        "beta": getattr(settings, "PAPER_HYBRID_BETA", None),
    },
    "dataset_hybrid_weights": {
        "alpha": getattr(settings, "DATASET_HYBRID_ALPHA", None),
        "beta": getattr(settings, "DATASET_HYBRID_BETA", None),
    },
}

for idx, test_case in enumerate(tqdm(all_test_cases, desc="평가 진행")):
    try:
        dataset_id = test_case['input_dataset']['svc_id']
        category = test_case['category']

        print(f"\n[{idx+1}/{len(all_test_cases)}] {test_case['input_dataset']['title']}")
        print(f"   카테고리: {category}")

        # 추천 생성
        recommendation_result = await agent.recommend(
            dataset_id,
            num_paper_recommendations=num_paper_recommendations,
            num_dataset_recommendations=num_dataset_recommendations
        )

        if 'error' in recommendation_result:
            print(f"   ❌ 오류: {recommendation_result['error']}")
            continue

        # ===== 추천 원본 결과 수집 =====
        # source_dataset
        src_ds = test_case.get("input_dataset", {})
        source_dataset_block = {
            "id": src_ds.get("svc_id"),
            "title": src_ds.get("title"),
            "description": src_ds.get("description"),
            "keywords": src_ds.get("keywords", []),
        }

        # search_result
        search_result = recommendation_result.get("search_result", {})
        search_result_block = {
            "paper_keywords": search_result.get("paper_keywords", []),
            "dataset_keywords": search_result.get("dataset_keywords", []),
            "paper_search_details": search_result.get("paper_search_details", []),
            "dataset_search_details": search_result.get("dataset_search_details", []),
        }

        # paper/dataset recommendations
        paper_reco = recommendation_result.get("paper_recommendations", [])
        dataset_reco = recommendation_result.get("dataset_recommendations", [])

        # candidates_analyzed
        candidates_analyzed = recommendation_result.get(
            "candidates_analyzed",
            (len(recommendation_result.get("paper_candidates", [])) +
             len(recommendation_result.get("dataset_candidates", []))
             if ("paper_candidates" in recommendation_result or "dataset_candidates" in recommendation_result)
             else (len(paper_reco) + len(dataset_reco)))
        )

        # 처리 시간
        processing_time_ms = recommendation_result.get("processing_time_ms", None)

        # 하나의 케이스 레코드 구성
        reco_record = {
            "source_dataset": source_dataset_block,
            "search_result": search_result_block,
            "paper_recommendations": paper_reco,
            "dataset_recommendations": dataset_reco,
            "processing_time_ms": processing_time_ms,
            "candidates_analyzed": candidates_analyzed,
            "model_info": _model_info_safe,
            "embedding_model_info": _embedding_info_safe,
        }
        all_recommendations.append(reco_record)
        # ==============================================

        # 평가 형식으로 변환
        predictions = []
        for paper in recommendation_result.get('paper_recommendations', []):
            predictions.append({
                'type': 'paper',
                'id': paper['id'],
                'rank': paper.get('rank'),
                'score': paper.get('score')
            })

        for dataset in recommendation_result.get('dataset_recommendations', []):
            predictions.append({
                'type': 'dataset',
                'id': dataset['id'],
                'rank': dataset.get('rank'),
                'score': dataset.get('score')
            })

        # 평가
        ground_truth = test_case['candidate_pool']
        eval_result = evaluate_recommendations(predictions, ground_truth, k_values)

        # 메타데이터 추가
        eval_result['test_case_index'] = idx
        eval_result['dataset_id'] = dataset_id
        eval_result['category'] = category
        eval_result['input_title'] = test_case['input_dataset']['title']
        eval_result['processing_time_ms'] = processing_time_ms if processing_time_ms is not None else recommendation_result.get('processing_time_ms', 0)
        # 선택: 추천 개수 기록(분석 편의)
        eval_result['num_predicted_papers'] = len([p for p in predictions if p['type'] == 'paper'])
        eval_result['num_predicted_datasets'] = len([p for p in predictions if p['type'] == 'dataset'])
        all_eval_results.append(eval_result)

        # 결과 요약 출력 (k_values 반영)
        summary_chunks = []
        for k in k_values:
            ndcg = eval_result.get(f"overall_ndcg@{k}", 0)
            mrr = eval_result.get(f"overall_mrr@{k}", 0)
            recall = eval_result.get(f"overall_recall@{k}", 0)
            summary_chunks.append(
                f"nDCG@{k}={ndcg:.4f}, MRR@{k}={mrr:.4f}, Recall@{k}={recall:.4f}"
            )
        print("   ✅ " + " | ".join(summary_chunks))

    except Exception as e:
        print(f"   ❌ 예외 발생: {e}")
        logging.error(f"테스트 케이스 {idx} 평가 실패: {e}", exc_info=True)
        continue

print("\n" + "=" * 80)
print(f"✅ 평가 완료: {len(all_eval_results)}/{len(all_test_cases)}개 케이스 성공")

🔍 13개 테스트 케이스 평가 시작
📏 평가 k 값: [3, 5]


평가 진행:   0%|          | 0/13 [00:00<?, ?it/s]


[1/13] 한반도 기상 관측 데이터 세트
   카테고리: 국내
2025-10-16 07:23:00,285 - src.agents.recommendation_agent - INFO - 추천 프로세스 시작: 데이터셋 ID c1f03ab868f32371ec97a650bcdf39ac
2025-10-16 07:23:00,393 - httpx - INFO - HTTP Request: GET https://dataon.kisti.re.kr/rest/api/search/dataset/c1f03ab868f32371ec97a650bcdf39ac?key=4936BC43D48603524DEDA2E2D56D6B46 "HTTP/1.1 200 200"
2025-10-16 07:23:00,395 - src.clients.dataon_client - INFO - Successfully retrieved metadata for dataset c1f03ab868f32371ec97a650bcdf39ac
2025-10-16 07:23:00,396 - src.clients.dataon_client - INFO - API Response:
{
  "response": {
    "elapsed time": "32 ms",
    "status": "200",
    "message": "OK",
    "total count": "1",
    "type": "json"
  },
  "records": {
    "svc_id": "c1f03ab868f32371ec97a650bcdf39ac",
    "ctlg_type": "02",
    "dataset_type": "01",
    "ctlg_type_pc": "dataset",
    "dataset_type_pc": "국내",
    "dataset_pub_dt_pc": "2025",
    "dataset_access_type_pc": "공개",
    "file_yn_pc": "직접다운로드",
    "dataset_cc_license

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


2025-10-16 07:23:02,175 - src.agents.recommendation_agent - INFO - ✅ LLM 키워드 생성 완료
2025-10-16 07:23:02,177 - src.agents.recommendation_agent - INFO -    원본 키워드 (3개): ['센서데이터', '한반도', '기상']
2025-10-16 07:23:02,177 - src.agents.recommendation_agent - INFO -    → 데이터셋 검색 (3개): ['한반도', '기상', '관측']
2025-10-16 07:23:02,178 - src.agents.recommendation_agent - INFO -    → 논문 검색 (4개): ['Korea Weather', 'climate observation', 'meteorology', 'South Korea']
2025-10-16 07:23:02,178 - src.agents.recommendation_agent - INFO - 검색 쿼리 생성 완료: 데이터셋(3개), 논문(4개)
2025-10-16 07:23:02,335 - httpx - INFO - HTTP Request: GET https://dataon.kisti.re.kr/rest/api/search/dataset/?key=370D3EC3BBD8E5DCCC40807922CCEC12&query=%ED%95%9C%EB%B0%98%EB%8F%84&page=1&size=5 "HTTP/1.1 200 200"
2025-10-16 07:23:02,340 - src.clients.dataon_client - INFO - Search completed: 5 datasets for '한반도'
2025-10-16 07:23:02,474 - httpx - INFO - HTTP Request: GET https://dataon.kisti.re.kr/rest/api/search/dataset/?key=370D3EC3BBD8E5DCCC40807

...


Batches: 100%|██████████| 1/1 [00:00<00:00, 80.57it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 105.21it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 93.99it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 102.73it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 92.65it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 130.06it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 94.42it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 151.94it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 74.29it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 130.23it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 92.25it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 117.95it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 92.30it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 57.70it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 87.01it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 63.02it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 94.67it/s]

Batches: 100%|██████████| 1/1 [00:00<00:0

2025-10-16 07:29:42,184 - src.agents.recommendation_agent - INFO - 상위 20개 논문, 11개 데이터셋 순위 결정 완료
2025-10-16 07:29:42,185 - src.agents.recommendation_agent - INFO - LLM paper 추천 생성 시도 1/2


2025-10-16 07:29:42,186 - src.agents.recommendation_agent - INFO - ================================================================================
2025-10-16 07:29:42,186 - src.agents.recommendation_agent - INFO - LLM에게 전송하는 paper 프롬프트:
2025-10-16 07:29:42,187 - src.agents.recommendation_agent - INFO - # Task: Re-rank and Select Top 5 paper Recommendations
You are a research recommendation expert. Re-rank the candidates and select the top 5 most relevant items.
## Source Dataset:
Title: Police recorded crime trends in Victoria during the COVID-19 pandemic
Description: The Crime Statistics Agency (CSA) has released a research paper entitled Police-recorded crime trends in Victoria during the COVID-19 pandemic. The paper examines trends in criminal offences and family violence incidents to 30 June 2020, focusing on comparing the period from 1 April 2020 to 30 June 2020 with the same period in 2019.\\r\\nThis paper aims to examine whether and how the COVID-19 pandemic, associated behavio

평가 진행: 100%|██████████| 13/13 [07:10<00:00, 33.13s/it]

   ✅ nDCG@3=1.0000, MRR@3=1.0000, Recall@3=0.3333 | nDCG@5=1.0000, MRR@5=1.0000, Recall@5=0.5556

✅ 평가 완료: 13/13개 케이스 성공


## 5. 평가 결과 분석 및 저장

### 5.1 평가 결과 저장 디렉토리 설정

In [7]:
# 타임스탬프 기반 결과 디렉토리 생성
timestamp = datetime.now().strftime('%Y%m%d%H%M')
eval_output_dir = os.path.join(project_root, 'figures', 'evaluation_results', timestamp)
os.makedirs(eval_output_dir, exist_ok=True)

print(f"📂 평가 결과 저장 경로: {eval_output_dir}")

# 추천 원본 결과 저장 
try:
    recommend_path = os.path.join(eval_output_dir, 'recommend_result.json')
    with open(recommend_path, 'w', encoding='utf-8') as f:
        json.dump(all_recommendations, f, indent=2, ensure_ascii=False)
    print(f"💾 recommend_result.json 저장 완료 → {recommend_path}")
except Exception as e:
    print(f"⚠️ recommend_result.json 저장 실패: {e}")

📂 평가 결과 저장 경로: /home/infidea/backup-data/paper-reco-agent/figures/evaluation_results/202510160730
💾 recommend_result.json 저장 완료 → /home/infidea/backup-data/paper-reco-agent/figures/evaluation_results/202510160730/recommend_result.json


### 5.2 전체 메트릭 계산

In [8]:
if not all_eval_results:
    print("⚠️  평가 결과가 없습니다")
else:
    # DataFrame 변환
    df_results = pd.DataFrame(all_eval_results)

    print("\n" + "=" * 80)
    print("📊 평균 평가 메트릭")
    print("=" * 80)

    # 메트릭 요약 계산
    metrics_summary = {}

    for k in k_values:
        print(f"\n📏 k={k} 결과:")
        print("-" * 80)

        k_metrics = {}

        for metric_type in ['overall', 'paper', 'dataset']:
            type_label = {'overall': '전체', 'paper': '논문', 'dataset': '데이터셋'}[metric_type]
            print(f"\n  {type_label}:")

            type_metrics = {}
            for metric in ['ndcg', 'mrr', 'recall', 'precision']:
                col_name = f'{metric_type}_{metric}@{k}'
                if col_name in df_results.columns:
                    mean_val = df_results[col_name].mean()
                    std_val = df_results[col_name].std()
                    type_metrics[metric] = {'mean': mean_val, 'std': std_val}
                    print(f"    • {metric.upper():10s}: {mean_val:.4f} ± {std_val:.4f}")

            k_metrics[metric_type] = type_metrics

        metrics_summary[f'k{k}'] = k_metrics

    # 통계
    print("\n" + "=" * 80)
    print("📈 기본 통계")
    print("=" * 80)
    print(f"  평균 추천 논문 수: {df_results['num_predicted_papers'].mean():.2f}개")
    print(f"  평균 추천 데이터셋 수: {df_results['num_predicted_datasets'].mean():.2f}개")
    print(f"  평균 관련 논문 수: {df_results['num_relevant_papers'].mean():.2f}개")
    print(f"  평균 관련 데이터셋 수: {df_results['num_relevant_datasets'].mean():.2f}개")
    print(f"  평균 처리 시간: {df_results['processing_time_ms'].mean():.0f} ms")


📊 평균 평가 메트릭

📏 k=3 결과:
--------------------------------------------------------------------------------

  전체:
    • NDCG      : 0.8462 ± 0.3755
    • MRR       : 0.8462 ± 0.3755
    • RECALL    : 0.2895 ± 0.1564
    • PRECISION : 0.7436 ± 0.3886

  논문:
    • NDCG      : 0.8462 ± 0.3755
    • MRR       : 0.8462 ± 0.3755
    • RECALL    : 0.5744 ± 0.3137
    • PRECISION : 0.7436 ± 0.3886

  데이터셋:
    • NDCG      : 0.9192 ± 0.2765
    • MRR       : 0.9231 ± 0.2774
    • RECALL    : 0.7167 ± 0.2915
    • PRECISION : 0.8462 ± 0.2924

📏 k=5 결과:
--------------------------------------------------------------------------------

  전체:
    • NDCG      : 0.8346 ± 0.3727
    • MRR       : 0.8462 ± 0.3755
    • RECALL    : 0.3748 ± 0.1979
    • PRECISION : 0.5846 ± 0.3211

  논문:
    • NDCG      : 0.8462 ± 0.3755
    • MRR       : 0.8462 ± 0.3755
    • RECALL    : 0.7205 ± 0.4068
    • PRECISION : 0.5731 ± 0.3308

  데이터셋:
    • NDCG      : 0.9161 ± 0.2757
    • MRR       : 0.9231 ± 0.2774
    • REC

### 5.3 카테고리별 분석

In [9]:
if all_eval_results:
    print("\n" + "=" * 80)
    print("📊 카테고리별 메트릭 (k=5)")
    print("=" * 80)

    category_metrics = {}

    for category in sorted(df_results['category'].unique()):
        category_data = df_results[df_results['category'] == category]

        print(f"\n📁 {category} (n={len(category_data)}):")
        print("-" * 80)

        cat_metrics = {}
        for metric_type in ['overall', 'paper', 'dataset']:
            type_label = {'overall': '전체', 'paper': '논문', 'dataset': '데이터셋'}[metric_type]
            print(f"  {type_label}:")

            type_metrics = {}
            for metric in ['ndcg', 'mrr', 'recall', 'precision']:
                col_name = f'{metric_type}_{metric}@5'
                if col_name in category_data.columns:
                    mean_val = category_data[col_name].mean()
                    type_metrics[metric] = mean_val
                    print(f"    • {metric.upper():10s}: {mean_val:.4f}")

            cat_metrics[metric_type] = type_metrics

        category_metrics[category] = cat_metrics


📊 카테고리별 메트릭 (k=5)

📁 국내 (n=11):
--------------------------------------------------------------------------------
  전체:
    • NDCG      : 0.8046
    • MRR       : 0.8182
    • RECALL    : 0.3521
    • PRECISION : 0.5273
  논문:
    • NDCG      : 0.8182
    • MRR       : 0.8182
    • RECALL    : 0.6697
    • PRECISION : 0.5136
  데이터셋:
    • NDCG      : 0.9025
    • MRR       : 0.9091
    • RECALL    : 0.8788
    • PRECISION : 0.6818

📁 해외 (n=2):
--------------------------------------------------------------------------------
  전체:
    • NDCG      : 1.0000
    • MRR       : 1.0000
    • RECALL    : 0.5000
    • PRECISION : 0.9000
  논문:
    • NDCG      : 1.0000
    • MRR       : 1.0000
    • RECALL    : 1.0000
    • PRECISION : 0.9000
  데이터셋:
    • NDCG      : 0.9908
    • MRR       : 1.0000
    • RECALL    : 0.7750
    • PRECISION : 0.8000


### 5.4 최종 결과 저장

**📁 저장되는 파일:**
1. **`EVALUATION_SUMMARY.txt`** ⭐ - 가장 먼저 봐야 할 요약 리포트
2. **`detailed_results.csv`** - 엑셀에서 열어볼 수 있는 상세 데이터
3. **`config.json`** - 실험 재현을 위한 설정 정보
4. **`metrics.json`** - 프로그램으로 읽을 수 있는 메트릭 데이터

In [10]:
if all_eval_results:
    print("\n💾 결과 저장 중...\n")

    # 1. 요약 리포트 생성 (가장 중요!)
    report_lines = []
    report_lines.append("=" * 80)
    report_lines.append("평가 요약 리포트")
    report_lines.append("=" * 80)
    report_lines.append("")
    report_lines.append(f"📅 평가 시각: {timestamp}")
    report_lines.append(f"🤖 LLM 모델: {settings.MODEL_NAME}")
    report_lines.append(f"🔍 임베딩 모델: {settings.EMBEDDING_MODEL}")
    report_lines.append(f"📊 테스트 케이스: {len(all_eval_results)}/{len(all_test_cases)} 성공")
    report_lines.append("")
    report_lines.append("=" * 80)
    report_lines.append("주요 성능 지표 (k=5)")
    report_lines.append("=" * 80)
    report_lines.append("")

    # k=5 전체 메트릭
    for metric in ['ndcg', 'mrr', 'recall', 'precision']:
        col_name = f'overall_{metric}@5'
        if col_name in df_results.columns:
            mean_val = df_results[col_name].mean()
            report_lines.append(f"  {metric.upper():10s} @5: {mean_val:.4f}")

    report_lines.append("")
    report_lines.append("=" * 80)
    report_lines.append("타입별 성능 (k=5)")
    report_lines.append("=" * 80)

    for metric_type in ['paper', 'dataset']:
        type_label = {'paper': '논문', 'dataset': '데이터셋'}[metric_type]
        report_lines.append("")
        report_lines.append(f"[{type_label}]")
        for metric in ['ndcg', 'mrr', 'recall', 'precision']:
            col_name = f'{metric_type}_{metric}@5'
            if col_name in df_results.columns:
                mean_val = df_results[col_name].mean()
                report_lines.append(f"  {metric.upper():10s}: {mean_val:.4f}")

    report_lines.append("")
    report_lines.append("=" * 80)
    report_lines.append("카테고리별 nDCG@5")
    report_lines.append("=" * 80)
    report_lines.append("")

    for category in sorted(df_results['category'].unique()):
        category_data = df_results[df_results['category'] == category]
        mean_val = category_data['overall_ndcg@5'].mean()
        report_lines.append(f"  {category:30s}: {mean_val:.4f} (n={len(category_data)})")

    report_lines.append("")
    report_lines.append("=" * 80)
    report_lines.append("전체 메트릭 (모든 k 값)")
    report_lines.append("=" * 80)

    for k in k_values:
        report_lines.append("")
        report_lines.append(f"k={k}:")
        report_lines.append("-" * 40)
        for metric_type in ['overall', 'paper', 'dataset']:
            type_label = {'overall': '전체', 'paper': '논문', 'dataset': '데이터셋'}[metric_type]
            report_lines.append(f"  [{type_label}]")
            for metric in ['ndcg', 'mrr', 'recall', 'precision']:
                col_name = f'{metric_type}_{metric}@{k}'
                if col_name in df_results.columns:
                    mean_val = df_results[col_name].mean()
                    std_val = df_results[col_name].std()
                    report_lines.append(f"    {metric.upper():10s}: {mean_val:.4f} ± {std_val:.4f}")

    report_lines.append("")
    report_lines.append("=" * 80)

    # 요약 리포트 저장
    summary_path = os.path.join(eval_output_dir, 'EVALUATION_SUMMARY.txt')
    with open(summary_path, 'w', encoding='utf-8') as f:
        f.write('\n'.join(report_lines))
    print(f"✅ 1. 요약 리포트 저장: EVALUATION_SUMMARY.txt")

    # 2. 상세 CSV 저장
    csv_path = os.path.join(eval_output_dir, 'detailed_results.csv')
    df_results.to_csv(csv_path, index=False, encoding='utf-8-sig')
    print(f"✅ 2. 상세 결과 CSV 저장: detailed_results.csv")

    # 3. 메트릭 JSON 저장
    all_metrics = {
        'summary': metrics_summary,
        'by_category': category_metrics,
        'individual_results': all_eval_results
    }

    metrics_path = os.path.join(eval_output_dir, 'metrics.json')
    with open(metrics_path, 'w', encoding='utf-8') as f:
        json.dump(all_metrics, f, indent=2, ensure_ascii=False)
    print(f"✅ 3. 메트릭 JSON 저장: metrics.json")

    print(f"\n" + "=" * 80)
    print(f"📂 모든 결과가 저장되었습니다:")
    print(f"   {eval_output_dir}")
    print(f"" + "=" * 80)
    print(f"\n⭐ 결과 확인: EVALUATION_SUMMARY.txt 파일을 먼저 읽어보세요!")

    # 요약 리포트 미리보기
    print(f"\n" + "=" * 80)
    print("📄 요약 리포트 미리보기:")
    print("=" * 80)
    print('\n'.join(report_lines[:50]))  # 처음 50줄만 표시
    if len(report_lines) > 50:
        print("\n... (전체 내용은 EVALUATION_SUMMARY.txt 참고) ...")
else:
    print("⚠️  저장할 결과가 없습니다")


💾 결과 저장 중...

✅ 1. 요약 리포트 저장: EVALUATION_SUMMARY.txt
✅ 2. 상세 결과 CSV 저장: detailed_results.csv
✅ 3. 메트릭 JSON 저장: metrics.json

📂 모든 결과가 저장되었습니다:
   /home/infidea/backup-data/paper-reco-agent/figures/evaluation_results/202510160730

⭐ 결과 확인: EVALUATION_SUMMARY.txt 파일을 먼저 읽어보세요!

📄 요약 리포트 미리보기:
평가 요약 리포트

📅 평가 시각: 202510160730
🤖 LLM 모델: google/gemma-2-9b-it
🔍 임베딩 모델: intfloat/multilingual-e5-large
📊 테스트 케이스: 13/13 성공

주요 성능 지표 (k=5)

  NDCG       @5: 0.8346
  MRR        @5: 0.8462
  RECALL     @5: 0.3748
  PRECISION  @5: 0.5846

타입별 성능 (k=5)

[논문]
  NDCG      : 0.8462
  MRR       : 0.8462
  RECALL    : 0.7205
  PRECISION : 0.5731

[데이터셋]
  NDCG      : 0.9161
  MRR       : 0.9231
  RECALL    : 0.8628
  PRECISION : 0.7000

카테고리별 nDCG@5

  국내                            : 0.8046 (n=11)
  해외                            : 1.0000 (n=2)

전체 메트릭 (모든 k 값)

k=3:
----------------------------------------
  [전체]
    NDCG      : 0.8462 ± 0.3755
    MRR       : 0.8462 ± 0.3755

... (전체 내용은 EVALUATION_SUMM

## 6. 평가 완료

**결과 파일 안내**
- 평가 결과는 `figures/evaluation_results/{timestamp}/` 디렉토리에 저장됩니다.

**파일 읽는 순서:**
1. `EVALUATION_SUMMARY.txt` ⭐⭐⭐
   - 가장 먼저 읽어야 할 파일
   - 주요 성능 지표와 카테고리별 결과 요약
   - 텍스트 에디터로 바로 열어볼 수 있음

2. `detailed_results.csv`
   - 엑셀이나 스프레드시트로 열어서 상세 분석
   - 각 테스트 케이스별 메트릭 포함
   - 필터링, 정렬, 차트 생성 가능

3. `recommend_result.json`
   - 평가 데이터 추천 결과

4. `metrics.json`
   - 프로그램으로 읽을 수 있는 메트릭 데이터
   - 자동화된 분석이나 비교에 사용

**주요 메트릭 설명**
- nDCG@k: 순위를 고려한 추천 품질 (0~1, 높을수록 좋음)
- MRR@k: 첫 번째 관련 항목의 순위 (0~1, 높을수록 좋음)
- Recall@k: 관련 항목 중 추천된 비율 (0~1, 높을수록 좋음)
- Precision@k: 추천 항목 중 관련된 비율 (0~1, 높을수록 좋음)